In [1]:
# Import needed librabtris
import os
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [22]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

In [23]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "timestamp": x["timestamp"],
})
movies = movies.map(lambda x: x["movie_title"])

In [25]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1000).map(
    lambda x: x["user_id"]))))

In [29]:
from locale import normalize


class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.user_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_tokens = False),
            tf.keras.layers.Embedding(len(unique_user_ids) +1, 32)
        ])

        self.timestamp_embedding = tf.keras.Sequential([
            tf.keras.layers.Discretization(timestamp_buckets.tolist()),
            tf.keras.layers.Embedding(len(timestamp_buckets)+1 , 32)
        ])

        self.normalized_timestamps = tf.keras.layers.Normalization()

        self.normalized_timestamps.adapt(timestamps)

    def call(self, inputs):
        if not self._use_timestamps:
            return self.user_embeddings(inputs["user_id"])

        return tf.concat([
            self.user_embedding(inputs["user_id"]),
            self.timestamp_embedding(inputs["timestamp"]),
            self.normalized_timestamp(inputs["timestamp"])
        ], axis = 1)

SyntaxError: invalid syntax (3898322551.py, line 21)